In [74]:
import geopandas as gpd
import pandas as pd

In [75]:
%%bash
<comunieventilink0.js sed 's/var json_comunieventilink0 = //g'  > comunieventilink0.geojson

In [76]:
%%bash
curl -sL "https://docs.google.com/spreadsheets/d/e/2PACX-1vQHIcaQrXHke2rFSGnydDo6dextEBaxrRL6lbSBNkR2OrQiKje25bOzv7cUsxiLQXSMwXqBkgL94y0W/pub?gid=0&single=true&output=csv" \
> richieste.csv

In [77]:
df = "comunieventilink0.geojson"
df = gpd.read_file(df)
df.head(3)

,cod_pro,cod_reg,comune,geometry,id,link,numero_eventi,pro_com
0,Salerno,Campania,Laureana Cilento,"POLYGON ((15.017164 40.324587, 15.016716 40.32...",0,<a href='http://www.foiapop.it/api/accesso-civ...,1,65060
1,Roma,Lazio,Fiumicino,"POLYGON ((12.238728 42.017981, 12.238873 42.01...",1,<a href='http://www.foiapop.it/api/accesso-civ...,4,58120
2,Latina,Lazio,Aprilia,"POLYGON ((12.643742 41.666867, 12.644387 41.66...",2,<a href='http://www.foiapop.it/api/accesso-civ...,1,59001


In [78]:
richieste=pd.read_csv('richieste.csv')
colonne=['Comune','Regione']
richieste=richieste[colonne]
richieste.head(3)

,Comune,Regione
0,Gangi,Sicilia
1,Velletri,Lazio
2,Messina,Sicilia


In [79]:
! rm 'merge.geojson'

In [80]:
merge=pd.merge(df,richieste,left_on="comune",right_on="Comune",how="left")
merge=merge.rename(columns = {'Comune':'richiesta'})
mergedf = gpd.GeoDataFrame(merge, geometry='geometry')
mergedf.richiesta.replace({r'^.+$' : r'sì'}, regex=True,inplace=True)
mergedf.richiesta = mergedf.richiesta.fillna('no')
mergedf.drop(['Regione'], axis=1, inplace=True)
mergedf.to_file('merge.geojson',driver='GeoJSON')

In [81]:
%%bash
cp "comunieventilink0.js" "old_comunieventilink0.js"
<merge.geojson sed  '1s/^/var json_comunieventilink0 = /' > "comunieventilink0.js"